In [ ]:
import numpy as np

e = 44
R = e /(2*np.sin(np.pi/5))
h = 0.1 * np.sqrt(50-10*np.sqrt(5)) * e

hr_ratio = h / R

print(f"R={R}")
print(f"h={h}")
print(f"hr_ratio={hr_ratio}")

# Load and plot bipyramid from STL file

In [ ]:
%matplotlib widget

import numpy as np

import CDEF

import scipy.optimize
from scipy.optimize import root, Bounds

import time

#Plotting
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker as ticker
from matplotlib.ticker import ScalarFormatter, NullFormatter

# Function to make 3D axes equal
def set_axes_equal(ax):
    """Make axes of 3D plot have equal scale so that spheres appear as spheres."""
    limits = np.array([ax.get_xlim3d(), ax.get_ylim3d(), ax.get_zlim3d()])
    centers = np.mean(limits, axis=1)
    max_range = 0.5 * np.max(limits[:,1] - limits[:,0])
    for center, setter in zip(centers, [ax.set_xlim3d, ax.set_ylim3d, ax.set_zlim3d]):
        setter(center - max_range, center + max_range)


hr_ratio = 0.1 * np.sqrt(50-10*np.sqrt(5))

In [ ]:
# Compute single-particle scattering curve
# number of scatterers
N = 30000

# generates 3D triangular mesh out of stl-files
# ideal cube
test_mesh = CDEF.read_stl('stl_files_examples/pentagonal_bipyramid.stl')

# cloud of punctiform scatterers based on mesh 
# from Halton series. Possible other values: sobol, random
#
test_cloud = CDEF.stl_cloud(test_mesh, N, sequence='halton')
test_cloud_random = CDEF.stl_cloud(test_mesh, N, sequence='random')

# single-particle scattering curve
# rbins - number of bins
# 
test_curve = CDEF.scattering_mono(test_cloud, selfcorrelation=True)
test_curve_random = CDEF.scattering_mono(test_cloud_random, selfcorrelation=True)
#
#no selfcorrelation
test_curve_random_no_selfcorr = CDEF.scattering_mono(test_cloud_random, selfcorrelation=False)


fig, ax = plt.subplots()
ax.loglog(test_curve[:,0], test_curve[:,1], label = "Halton")
ax.loglog(test_curve_random[:,0], test_curve_random[:,1], label = "true-random")
ax.loglog(test_curve_random_no_selfcorr[:,0], test_curve_random_no_selfcorr[:,1], '--', label = "true-random, no selfcorr.")
ax.legend()

#Visualization of point cloud
fig = plt.figure(figsize=(8,6)) 

ax = fig.add_subplot(111, projection='3d')

ax.plot(test_cloud.T[0], test_cloud.T[1], test_cloud.T[2], "r.", alpha=0.2) 

ax.view_init(30, 30)
set_axes_equal(ax)  # <-- apply equal aspect
plt.show()

# Rounded edges 0.1

In [ ]:
# Compute single-particle scattering curve
# number of scatterers
N = 30000

# generates 3D triangular mesh out of stl-files
# ideal cube
test_mesh = CDEF.read_stl('stl_files_examples/pentagonal_bipyramid_all_edges_0.10-Körper.stl')

# cloud of punctiform scatterers based on mesh 
# from Halton series. Possible other values: sobol, random
#
test_cloud = CDEF.stl_cloud(test_mesh, N, sequence='halton')
test_cloud_random = CDEF.stl_cloud(test_mesh, N, sequence='random')

# single-particle scattering curve
# rbins - number of bins
# 
test_curve = CDEF.scattering_mono(test_cloud, selfcorrelation=True)
test_curve_random = CDEF.scattering_mono(test_cloud_random, selfcorrelation=True)
#
#no selfcorrelation
test_curve_random_no_selfcorr = CDEF.scattering_mono(test_cloud_random, selfcorrelation=False)


fig, ax = plt.subplots()
ax.loglog(test_curve[:,0], test_curve[:,1], label = "Halton")
ax.loglog(test_curve_random[:,0], test_curve_random[:,1], label = "true-random")
ax.loglog(test_curve_random_no_selfcorr[:,0], test_curve_random_no_selfcorr[:,1], '--', label = "true-random, no selfcorr.")
ax.legend()

#Visualization of point cloud
fig = plt.figure(figsize=(8,6)) 

ax = fig.add_subplot(111, projection='3d')

ax.plot(test_cloud.T[0], test_cloud.T[1], test_cloud.T[2], "r.", alpha=0.2) 

ax.view_init(30, 30)
set_axes_equal(ax)  # <-- apply equal aspect
plt.show()

# Create bipyramid with python

In [2]:
from build123d import export_stl
from ocp_vscode import show
import numpy as np

def bipyramid(R, h, base_corners=5, fillet_radius=0.1, center=(0,0,0), rotation=0.0):
    from build123d import Solid, Shell, Face, Wire, export_stl, fillet
    from scipy.spatial import ConvexHull
    import numpy as np

    def bipyramid_vertices(R, h, base_corners=5, center=(0,0,0), rotation=0.0):
        """Generate vertices of a bipyramid with a variable amount of base corners.

        Args:
            R (float): radius of the base.
            h (float): height of the apices above and below the base plane.
            center (tuple): (x, y, z) coordinates of the bipyramid center.
            rotation (float): rotation angle in radians applied to the base pentagon.

        Returns:
            List of 7 tuples [(x,y,z), ...] representing the bipyramid vertices.
        """
        cx, cy, cz = center
        vertices = []
        
        # 5 vertices of pentagonal base in XY plane with rotation
        for i in range(base_corners):
            angle = 2 * np.pi * i / base_corners + rotation
            x = cx + R * np.cos(angle)
            y = cy + R * np.sin(angle)
            z = cz
            vertices.append((x, y, z))
        
        # apex vertex at +h and -h
        vertices.append((cx, cy, cz + h))
        vertices.append((cx, cy, cz - h))
        
        return vertices

    vertices = bipyramid_vertices(R=R, h=h, base_corners=base_corners, center=center, rotation=rotation)

    # Create a convex hull from the vertices
    hull = ConvexHull(vertices).simplices.tolist()

    # Create faces from the vertex indices
    faces = []
    for face_vertex_indices in hull:
        corner_vertices = [vertices[i] for i in face_vertex_indices]
        faces.append(Face(Wire.make_polygon(corner_vertices)))

    # Create the solid from the Faces
    bipy = Solid(Shell(faces)).clean()
    return fillet(bipy.edges(), radius=fillet_radius)


base_corners = 5
side_length = 1
height = side_length * 0.1 * np.sqrt(50-10*np.sqrt(5))
radius = side_length / (2*np.sin(np.pi/base_corners))
fillet_radius = radius * 0.1

bipy = bipyramid(R=radius, h=height, base_corners=base_corners, fillet_radius=fillet_radius)

show(bipy, port=3939)

export_stl(bipy, "stl/bipy.stl")


c


True